In [2]:
import numpy as np
import parmed as pmd
from utils import base_units

# MosDef packages
import foyer
from foyer import Forcefield
#import hoomd
import mbuild as mb



In [3]:
# User inputs

In [34]:
def compound(smiles_string):
    
    comp_mb = mb.load(smiles_string, smiles = True) # mbuild compound
    comp_pmd = comp_mb.to_parmed() #parmed compound
    comp_mass = np.sum([atom.mass for atom in comp_pmd.atoms]) # Mass of the compound
    comp_mb.mass = comp_mass # Update mb compound's mass

    GAFF = foyer.forcefields.load_GAFF()
    comp_typed = GAFF.apply(comp_mb, assert_dihedral_params=False)
    for atom_pmd, atom_mb in zip(comp_typed, comp_mb):
        atom_mb.name = "{}".format(atom_pmd.type)
    atom_types = [p.name for p in comp_mb.particles()]
    
    return comp_mb, atom_types

def build_box(compound, num_compounds):
    system_box = mb.packing.fill_box(compound, num_compounds, box=mb.Box([1,1,1]))
    return system_box

In [39]:
comp = compound('c1ccccc1')
comp, comp_types = compound('c1sccc1CCCCCC')
sys_box = build_box(comp, 10)
sys_box.visualize()

/home/chris/cme/mbuild/mbuild/compound.py:2443: UserWarning: Guessing that "<cc_r5 pos=( 0.0046, 0.1164,-0.0004), 0 bonds, id: 139737035428304>" is element: "C"
  atom, element))
/home/chris/cme/mbuild/mbuild/compound.py:2443: UserWarning: Guessing that "<ss pos=(-0.1117,-0.0083,-0.0007), 0 bonds, id: 139737035430672>" is element: "S"
  atom, element))
/home/chris/cme/mbuild/mbuild/compound.py:2443: UserWarning: Guessing that "<c3 pos=( 0.2593, 0.1540, 0.0004), 0 bonds, id: 139737035431696>" is element: "C"
  atom, element))
/home/chris/cme/mbuild/mbuild/compound.py:2443: UserWarning: Guessing that "<h4 pos=(-0.0290, 0.2193,-0.0007), 0 bonds, id: 139737035430608>" is element: "H"
  atom, element))
/home/chris/cme/mbuild/mbuild/compound.py:2443: UserWarning: Guessing that "<ha pos=( 0.2266,-0.1337, 0.0006), 0 bonds, id: 139737042881808>" is element: "H"
  atom, element))
/home/chris/cme/mbuild/mbuild/compound.py:2443: UserWarning: Guessing that "<hc pos=( 0.3198, 0.1286,-0.0876), 0 bond

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

In [29]:
'''
These steps below are essentially trying to recreate the process in planckton/placnkton/init.py
The goal is to start with a smiles string rather than an already typed .mol2 file
'''
# COMPOUND: 

# Set up compound
p3ht_smi = 'c1sccc1CCCCCC'
comp_mb = mb.load(p3ht_smi, smiles = True) # mbuild compound

# Find compound mass
comp_pmd = comp_mb.to_parmed() #parmed compound
comp_mass = np.sum([atom.mass for atom in comp_pmd.atoms]) # Mass of the compound
comp_mb.mass = comp_mass # Update mb compound's mass

# Type compound
GAFF = foyer.forcefields.load_GAFF()
comp_pmd_typed =  GAFF.apply(comp_mb, assert_dihedral_params=False) # parmed structure
#comp_mb_typed = mb.Compound() # create new mbuild compound
#comp_mb_typed.from_parmed(comp_pmd_typed) # Load from the typed parmed structure
for atom_pmd, atom_mb in zip(comp_pmd_typed, comp_mb):
    atom_mb.name = "{}".format(atom_pmd.type)  # Renaming mb comp atom names from atom types of parmed compound
    atom_mb.pos /= 10